# Analysing Star Wars

by `Nurseiit Abdimomyn`

id `20172001`

at `nurs@unist.ac.kr`

In [2]:
import csv # here we go again :P
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# all episodes
episodes = {}

for i in range(1, 7):
    with open('data/StarWars_ep%d.csv' % i, 'r') as f:
        reader = csv.reader(f)
        ep = [event for event in reader][1:] # ignore header
        episodes[i] = ep

In [13]:
actors = {}

for event in episodes[1]:
    sequence = int(event[2])
    actors[sequence] = event[5]

all_actors = list(set([actor[1] for actor in actors.items()]))
all_actors

['FIGHTER PILOT',
 'RUNE',
 'KI-ADI',
 'ANAKIN',
 'ANAKNI',
 'DARTH MAUL',
 'CAPTAIN',
 'QU-IG0N',
 'KITSTER',
 'GENERAL CEEL',
 'GREEDO',
 'EG-9',
 'BOSS NASS',
 'PR-4',
 'JABBA',
 'MACE WINDU',
 'LOTT DOD',
 'AMIDALA',
 'BAIL ORGANA',
 'AKS MOE',
 'BATTLE DROID 3B3',
 'DROID GUARD',
 'CAPT. PANAKA',
 'GUARD',
 'TC14',
 'MAD AMEDDA',
 'ANAKN',
 'GUARD DROID',
 'SEBULBA',
 'BATTLE DROID CAPTAIN',
 'CAPT. TARPALS',
 'PALPATINE',
 'YODA',
 'DARTH SIDIOUS',
 'OWO-1',
 'VALORUM',
 'JAR JAR',
 'OBI-WAN',
 'SEEK',
 'RIC OLIE',
 'QUI-GON',
 'GUI-GON',
 'RABE',
 'NUTE',
 'DOFINE',
 'PADME',
 'CAPTAIN PANAKA',
 'TC-14',
 'BRAVO THREE',
 'GUNGAN LOOKOUT',
 'A',
 'ANKAIN',
 'AMEE',
 'PILOT',
 'SABE',
 'PANAKA',
 'SHMI',
 'WATTO',
 'TEY HOW',
 'PK-4',
 'VENDOR',
 'BIBBLE',
 'OOM-9',
 'JIRA',
 'MAS AMEDDA',
 'ASSEMBLY',
 'THREEPIO',
 'P-59',
 'PALAPATINE',
 'FANTA',
 'BRAVO TWO',
 'WALD']